In [7]:
import datetime
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from chart import *
from table import *
from document import *

In [8]:
# 메타 정보

today = datetime.date.today()

y = today.year
m = today.month
d = today.day
w = today.weekday()

# report version
rtype = '월간' # 연간, 반기
month = '6월' # 7월
region = 'All' # All|KR|CN|NA|EU|SG|RU
v = '1'

if region == 'KR':
    regex = '^(KR|Global|All)' #' KR'
elif region == 'CN':
    regex = '^(CN|Global|All)'
elif region == 'NA':
    regex = '^(NA|Global|All)'
elif region == 'EU':
    regex = '^(EU|Global|All)'
elif region == 'SG':
    regex = '^(SG|Global|All)'
elif region == 'RU':
    regex = '^(SG|Global|All)'
else:
    regex = '^(KR|CN|NA|EU|SG|RU|Global|All)'

print("보고서 유형: ", rtype, "월: ", month)
print(regex)

보고서 유형:  월간 월:  6월
^(KR|CN|NA|EU|SG|RU|Global|All)


In [9]:
# Document 기본 폰트
style = document.styles['Normal']
style.font.name = '맑은고딕'
style.font.size = Pt(12)
style._element.rPr.rFonts.set(qn('w:eastAsia'), '맑은고딕')

In [10]:
# Chart 기본 폰트
alt.themes.register('맑은고딕', hanfont)
alt.themes.enable('맑은고딕')

ThemeRegistry.enable('맑은고딕')

In [11]:
# Cover Page
from docxtpl import DocxTemplate

doc = DocxTemplate("cover-page-template.docx")
context = { 
    'Service' : "HMC CCS Infra 운영/관리",      
    'Title' : "Monthly Report",    
    'Region' : "Global",        
    'Year' : "2022",
    'Month' : "6",
    'Department' : "클라우드운영센터",
    'Author' : "정인환"
}
doc.render(context)
year = 2022
month = 6
doc.save(f"MR-{year}-{month}.docx")

if region == 'All':
    document.add_heading(f'[2022년.{month}] Cloud Infra 운영', level=0)
else:
    document.add_heading(f'[2022년.{month}] {region} Cloud Infra 운영', level=0)

document.add_heading(f'{y}.{m}.{d}', level=1)
document.add_heading(f'클라우드운영센터', level=1)
document.add_page_break()


## 요약

In [12]:
# ETL
base_path = r"./data/요약_202206_서비스관리_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)

# 리전 & 운영계, 월간보고서용
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

KeyError: 'There is no sheet named 6집계'

In [ ]:
# Chart
if region == 'All':
    # Data
    pivot, total = getPivotTable(df, month)
    source = pivot

    # Chart
    chart = getPieChart(source)
    chart.save(f'./charts/summary_{y}_{m}_{d}_{v}_1.png')

    # Docx
    document.add_paragraph('요약')    
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')
    document.add_picture(f'./charts/summary_{y}_{m}_{d}_{v}_1.png')
else:
    pass

In [ ]:
# Document
addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.1 모니터링(인시던트핸들링)

In [ ]:
base_path = r"./data/1_1_202206_모니터링_인시던트핸들링_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df['장애전파소요시간'] = (pd.to_datetime(df['장애전파시간.1'])-pd.to_datetime(df['발생인지시간'])).astype('timedelta64[m]')

# Pivoting
pivot, total = getPivotTable(df, month)

# 리전 & 운영계, 월간보고서용
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]
df_filtered, timeslot = getTables(df_filtered, regex, month)
    
# SLA failed
df_failed = df_filtered[ (df_filtered['장애전파'] > 10) & (df_filtered['월']== month)]
df_failed[['날짜', '조치내용', 'JIRA_Ticket_No', '장애전파소요시간']]

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 스캐터 차트    
chart = getScatterChart(timeslot, month)
chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_2.png')

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_3.png')

In [ ]:
# Document
document.add_paragraph('모니터링(인시던트핸들링)', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'장애전파 소요시간(분)', style='List Bullet')
document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_2.png')
## 평균 소요 시간, 최대 소요 시간

document.add_paragraph(f'장애전파시간 초과 이벤트', style='List Bullet')
addFailedTable(df_failed, regex, month)

document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_3.png')   

In [ ]:
# Document
document.add_page_break()

## 1.2 이슈 관리

In [ ]:
base_path = r"./data/1_2_202206_이슈관리_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/im_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/im_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('이슈 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/im_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/im_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
addTable4(df_filtered, regex, month)
document.add_page_break()

## 1.3 장애 관리

In [ ]:
base_path = r"./data/1_3_202206_장애관리_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/pm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/pm_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('장애(RCA) 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'6월: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/pm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/pm_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
addTable4(df_filtered, regex, month)
document.add_page_break()

## 1.4 변경 관리

In [ ]:
base_path = r"./data/1_4_202206_변경관리_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('변경 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
addTable3(df_filtered, regex, month)
document.add_page_break()

## 1.5 요청 관리

In [ ]:
base_path = r"./data/1_5_202206_요청관리_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('요청 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
# addTable3(df_filtered, regex, month)
document.add_page_break()

## 1.6 자산 관리 - VM

In [ ]:
base_path = r"./data/1_6_202206_자산관리_VM_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/vm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/vm_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('자산 관리(vm)', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/vm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 VM 수 (추세)', style='List Bullet')
document.add_picture(f'./charts/vm_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.6 자산 관리 - DB

In [ ]:
base_path = r"./data/1_6_202206_자산관리_DB_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df_filtered, month)
pivot2, total = getPivotTable2(df_filtered, month)
pivot2 = pivot2.reset_index()

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/db_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# source = pivot2
# chart = getPieChart2(source)
# chart.save(f'./charts/db_{region}_{y}_{m}_{d}_2.png')

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/db_{region}_{y}_{m}_{d}_3.png')

In [ ]:
# Document
document.add_paragraph('자산 관리(db)', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_1.png')
else:
    pass

addPivot2Table(pivot2)
# document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_2.png')
document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_3.png')   

In [ ]:
# Document
document.add_page_break()

## 1.6 자산 관리 - K8s

In [ ]:
base_path = r"./data/1_6_202206_자산관리_k8s_0727_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

In [44]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered)
    chart.save(f'./charts/k8s_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - Bar 차트
chart = getBarChart(df_filtered, month)
chart.save(f'./charts/k8s_{region}_{y}_{m}_{d}_2.png')

WARN Infinite extent for field "합계_start": [Infinity, -Infinity]
WARN Infinite extent for field "합계_end": [Infinity, -Infinity]
WARN Infinite extent for field "합계_start": [Infinity, -Infinity]
WARN Infinite extent for field "합계_end": [Infinity, -Infinity]


TypeError: getBarChart() takes 1 positional argument but 2 were given

In [41]:
# Document
document.add_paragraph('자산 관리(k8s)', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/k8s_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/k8s_{region}_{y}_{m}_{d}_2.png')   

In [42]:
# Document
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()

In [43]:
document.save('demo.docx')

## 1.7 용량 관리

In [49]:
base_path = r"./data/1_7_202206_용량관리_0720_01.ods"
sheet = f'{month}집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df, month)

In [50]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered)
    chart.save(f'./charts/cpm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/cpm_{region}_{y}_{m}_{d}_2.png')

WARN Infinite extent for field "합계_start": [Infinity, -Infinity]
WARN Infinite extent for field "합계_end": [Infinity, -Infinity]
WARN Infinite extent for field "합계_start": [Infinity, -Infinity]
WARN Infinite extent for field "합계_end": [Infinity, -Infinity]


ValueError: sum(count) encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

In [ ]:
# Document
document.add_paragraph('용량(이슈) 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/cpm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/cpm_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
addSummaryTable(df_filtered, regex, month)
document.add_page_break()

In [ ]:
document.add_paragraph(f'자원 현황표', style='List Bullet')

## 1.8 백업 관리

In [ ]:
base_path = r"./data/1_8_202206_백업관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered, month)
    chart.save(f'./charts/bm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/bm_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('백업 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/bm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/bm_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.9 보안 관리

In [ ]:
base_path = r"./data/1_9_202206_보안관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered, month)
    chart.save(f'./charts/sm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/sm_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('보안 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/sm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/sm_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.10 하드웨어 정기점검

In [ ]:
base_path = r"./data/1_10_202206_정기점검_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered, month)
    chart.save(f'./charts/rc_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/rc_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('정기점검', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/rc_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/rc_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## Save Document

In [52]:
document.save(f'{month}_{region}_월간보고서_{y}_{m}_{d}.docx')

## 보고서 표지 (커버 쉬트)

In [51]:
# Cover Page
from docxtpl import DocxTemplate

doc = DocxTemplate("cover-page-template.docx")
context = { 
    'Service' : "HMC CCS Infra 운영/관리",      
    'Title' : "Monthly Report",    
    'Region' : "Global",        
    'Year' : "2022",
    'Month' : "6",
    'Department' : "클라우드운영센터",
    'Author' : "정인환"
}
doc.render(context)
year = 2022
mon = 6
doc.save(f"MR-{year}-{mon}.docx")

In [53]:
# Merged Doc

from docx import Document

files = [f"MR-{year}-{mon}.docx", f'{month}_{region}_월간보고서_{y}_{m}_{d}.docx' ]

def combine_word_documents(files):
    merged_document = Document()

    for index, file in enumerate(files):
        sub_doc = Document(file)

        # Don't add a page break if you've reached the last file.
        if index < len(files)-1:
           sub_doc.add_page_break()

        for element in sub_doc.element.body:
            merged_document.element.body.append(element)

    merged_document.save(f'merged_{month}_{region}_월간보고서_{y}_{m}_{d}.docx')

combine_word_documents(files)